# install 목록

In [ ]:
!pip install opencv-python
!pip install dlib
!pip install numpy
!pip install tensorflow
!pip install keras

# import 먼저 해보고 numpy 다운그레이드 오류가 생길시 numpy삭제후 다음 실행

In [10]:
!pip uninstall numpy
!pip install "numpy<2"

Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Would remove:
    /opt/homebrew/bin/f2py
    /opt/homebrew/lib/python3.12/site-packages/numpy-1.26.4.dist-info/*
    /opt/homebrew/lib/python3.12/site-packages/numpy/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


# import

In [1]:
import cv2
import dlib
import numpy as np
from keras.models import load_model

c:\Users\gram\miniconda3\Lib\site-packages\h5py\__init__.py:36: UserWarning: h5py is running against HDF5 1.14.4 when it was built against 1.14.2, this may cause problems
  _warn(("h5py is running against HDF5 {0} when it was built against {1}, "


# main code

### 사전 데이터 로드

In [2]:
# 얼굴 인식
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# 표정 인식을 위한 눈, 코, 입등의 위치 반환
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# 표정 라벨링
expression_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# 표정 가중치 모델
model = load_model('emotion_model.hdf5', compile=False)


### 필터 클래스 추가 ( 여기서 작업하시면 될 것 같아요)

In [5]:
class EmotionFilters:
    def __init__(self):
        self.filters = {
            'Angry': self.angry_filter,        # 빨간색
            'Disgust': self.disgust_filter,    # 초록색
            'Fear': self.fear_filter,          # 보라색
            'Happy': self.happy_filter,        # 노란색
            'Sad': self.sad_filter,            # 파란색
            'Surprise': self.surprise_filter,  # 핑크색
            'Neutral': self.neutral_filter     # 중립은 변화 없음
        }

    def apply_filter(self, frame, expression_label):
        if expression_label in self.filters:
            frame = self.filters[expression_label](frame)
        return frame

    def colorize(self, frame, hue):
        hue = int(hue * 0.5)  # 360도 범위에서 180도 범위로 변환
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        hsv[:, :, 0] = hue
        hsv[:, :, 1] = 255  # 채도를 최대로 설정하여 색상 강조
        return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)


    def angry_filter(self, frame):
        return self.colorize(frame, 0)  # 빨간색

    def disgust_filter(self, frame):
        return self.colorize(frame, 120)  # 초록색

    def fear_filter(self, frame):
        return self.colorize(frame, 270)  # 보라색

    def happy_filter(self, frame):
        return self.colorize(frame, 60)  # 노란색

    def sad_filter(self, frame):
        return self.colorize(frame, 240)  # 파란색

    def surprise_filter(self, frame):
        return self.colorize(frame, 300)  # 핑크색
    
    def neutral_filter(self, frame):
        # 중립 표정일 때는 원본 그대로 반환
        return frame


## 비디오 캠 버전 - 표정 인식 연습용

In [ ]:
# 얼굴 인식
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# 표정 인식을 위한 눈, 코, 입등의 위치 반환
predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')

# 표정 라벨링
expression_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

# 표정 가중치 모델
model = load_model('emotion_model.hdf5', compile=False)
emotion_filters = EmotionFilters()

# 비디오 실행
video_capture = cv2.VideoCapture(0)

prev_faces = []

while True:
    # ret, frame 반환
    ret, frame = video_capture.read()
    
    if not ret:
        break

    # 얼굴인식을 위해 gray 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # 얼굴 인식
    # scaleFactor이 1에 가까울수록 표정 인식이 잘 되고 멀 수록 잘 안됨
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    
    #region 얼굴이 인식되면 표정을 인식
    for (x, y, w, h) in faces:
        # 얼굴 크기에 알맞도록 사각형 그리기
        cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)

        # 얼굴 크기 반환
        face_roi = gray[y:y+h, x:x+w]

        # 표정을 인식하기 위해 표정 dataset과 똑같은 사이즈 변환
        # dataset 이미지와 입력된 얼굴의 크기가 다르면 error 발생
        face_roi = cv2.resize(face_roi, (64, 64))
        face_roi = np.expand_dims(face_roi, axis=-1)
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = face_roi / 255.0

        # 모델을 통해 표정 분석
        output = model.predict(face_roi)[0]

        # 해당 표정의 값 반환
        expression_index = np.argmax(output)

        # 표정에 따른 label 값 저장
        expression_label = expression_labels[expression_index]
           # 표정에 맞는 필터 적용
        frame = emotion_filters.apply_filter(frame, expression_label)
        # print(expression_label, end=' ')
        # 표정 값 출력
        cv2.putText(frame, expression_label, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    #endregion
    
    # 출력
    cv2.imshow('Expression Recognition', frame)

    # esc 누를 경우 종료
    key = cv2.waitKey(25)
    if key == 27:
        break

video_capture.release()
cv2.destroyAllWindows()
# 밑에는 오류가 아닌 창이 안닫힐때 해결법 
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)


## 영상인식 버전 - 제출할 코드용

In [6]:
# 필터 클래스 초기화
emotion_filters = EmotionFilters()

prev_faces = []
# 저장된 비디오 파일 경로로 변경
video_capture = cv2.VideoCapture('example/emotion.mp4')  # 비디오 파일 경로 변경
# 출력
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # 프레임 크기 가져오기
    height, width = frame.shape[:2]

    # 얼굴인식을 위해 gray 변환
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # 얼굴 인식
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # 초기 표정 라벨 설정
    expression_label = 'No Face'  # 얼굴이 검출되지 않았을 때의 기본값

    for (x, y, w, h) in faces:
        # 얼굴 크기에 알맞도록 사각형 그리기 (갈색으로 변경)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (42, 42, 165), 2)

        # 얼굴 크기 반환
        face_roi = gray[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (64, 64))
        face_roi = np.expand_dims(face_roi, axis=-1)
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = face_roi / 255.0

        # 모델을 통해 표정 분석
        output = model.predict(face_roi)[0]
        expression_index = np.argmax(output)
        expression_label = expression_labels[expression_index]

        # 표정에 맞는 필터 적용
        frame = emotion_filters.apply_filter(frame, expression_label)

    # 오른쪽 하단에 표정 라벨 화면에 표시
    text_location = (width - 150, height - 20)  # 오른쪽 하단 조정
    cv2.putText(frame, expression_label, text_location, cv2.FONT_HERSHEY_SIMPLEX, 0.9, (42, 42, 165), 2)

    cv2.imshow('Expression Recognition with Filters', frame)

    # q누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()
# 맥의 경우 아래 waitkey를 반복해야 꺼짐 
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)
cv2.waitKey(1)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 370ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━

-1

### 배경 크로마키 적용시켜보기

### 필터

In [ ]:
class EmotionFilters:
    def __init__(self):
        self.filters = {
            'Angry': self.angry_filter,
            'Disgust': self.disgust_filter,
            'Fear': self.fear_filter,
            'Happy': self.happy_filter,
            'Sad': self.sad_filter,
            'Surprise': self.surprise_filter,
            'Neutral': self.neutral_filter
        }
        self.sad_video = cv2.VideoCapture('effect/sad_effect.mp4')
        self.surprise_video = cv2.VideoCapture('effect/surprise_effect.mp4')
        self.angry_video = cv2.VideoCapture('effect/angry_effect.mp4')
        self.happy_video = cv2.VideoCapture('effect/lovely_effect.mp4')

        self.sad_video_playing = False  
        self.surprise_video_playing = False 
        self.angry_video_playing = False  
        self.happy_video_playing = False

    def apply_filter(self, frame, expression_label):
        if expression_label in self.filters:
            return self.filters[expression_label](frame)
        return frame
    
    def colorize(self, frame, hue):
        # BGR 이미지를 HSV로 변환
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
        # hue 값에 따라 H 채널 조정, 0-179 사이의 값으로 제한
        hsv[:, :, 0] = hue % 180  # 값의 범위를 안전하게 조정
        # HSV 이미지를 BGR로 다시 변환
        return cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)


    def sad_filter(self, frame):
        if not self.sad_video_playing:
            # 슬픈 감정이 처음 감지되면 비디오를 처음부터 재생
            self.sad_video.set(cv2.CAP_PROP_POS_FRAMES, 0)
            self.sad_video_playing = True

        ret, sad_frame = self.sad_video.read()
        if not ret:
            # 비디오가 끝나면 재생 상태를 False로 설정
            self.sad_video_playing = False
            return frame  # 비디오가 끝나면 원본 프레임을 반환

        # 크로마키 처리
        hsv = cv2.cvtColor(sad_frame, cv2.COLOR_BGR2HSV)
        lower_green = np.array([40, 40, 40])
        upper_green = np.array([80, 255, 255])
        mask = cv2.inRange(hsv, lower_green, upper_green)
        sad_frame_no_bg = cv2.bitwise_and(sad_frame, sad_frame, mask=cv2.bitwise_not(mask))

        # 영상 합성
        return cv2.addWeighted(frame, 0.5, sad_frame_no_bg, 0.5, 0)

    def angry_filter(self, frame):
        if not self.angry_video_playing:
            self.angry_video.set(cv2.CAP_PROP_POS_FRAMES, 0)
            self.angry_video_playing = True

        ret, angry_frame = self.angry_video.read()
        if not ret:
            self.angry_video_playing = False
            return frame

        # 크로마키 처리: 초록색 배경 제거
        hsv = cv2.cvtColor(angry_frame, cv2.COLOR_BGR2HSV)
        lower_green = np.array([40, 40, 40])
        upper_green = np.array([80, 255, 255])
        mask = cv2.inRange(hsv, lower_green, upper_green)
        angry_frame_no_bg = cv2.bitwise_and(angry_frame, angry_frame, mask=cv2.bitwise_not(mask))

        # 영상 합성
        return cv2.addWeighted(frame, 0.5, angry_frame_no_bg, 0.5, 0)



    def disgust_filter(self, frame):
        return self.colorize(frame, 120)

    def fear_filter(self, frame):
        return self.colorize(frame, 270)

    def happy_filter(self, frame):
        if not self.happy_video_playing:
            self.happy_video.set(cv2.CAP_PROP_POS_FRAMES, 0)
            self.happy_video_playing = True

        ret, happy_frame = self.happy_video.read()
        if not ret:
            self.happy_video_playing = False
            return frame

        # 크로마키 처리: 검정색 배경 제거
        gray = cv2.cvtColor(happy_frame, cv2.COLOR_BGR2GRAY)
        _, mask = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)  # 검정색 부분을 제거
        happy_frame_no_bg = cv2.bitwise_and(happy_frame, happy_frame, mask=mask)

        # 영상 합성
        return cv2.addWeighted(frame, 0.5, happy_frame_no_bg, 0.5, 0)


    def surprise_filter(self, frame):
        if not self.surprise_video_playing:
            self.surprise_video.set(cv2.CAP_PROP_POS_FRAMES, 0)
            self.surprise_video_playing = True

        ret, surprise_frame = self.surprise_video.read()
        if not ret:
            self.surprise_video_playing = False
            return frame

        # 크로마키 처리
        hsv = cv2.cvtColor(surprise_frame, cv2.COLOR_BGR2HSV)
        lower_green = np.array([40, 40, 40])
        upper_green = np.array([80, 255, 255])
        mask = cv2.inRange(hsv, lower_green, upper_green)
        surprise_frame_no_bg = cv2.bitwise_and(surprise_frame, surprise_frame, mask=cv2.bitwise_not(mask))

        # 영상 합성
        return cv2.addWeighted(frame, 0.5, surprise_frame_no_bg, 0.5, 0)


    def neutral_filter(self, frame):
        return frame

### main code

In [ ]:
video_capture = cv2.VideoCapture('example/emotion.mp4')
emotion_filters = EmotionFilters()


def detect_emotion(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    if len(faces) == 0:
        return 'No Face'

    for (x, y, w, h) in faces:
        face_roi = gray[y:y+h, x:x+w]
        face_roi = cv2.resize(face_roi, (64, 64))
        face_roi = np.expand_dims(face_roi, axis=-1)
        face_roi = np.expand_dims(face_roi, axis=0)
        face_roi = face_roi / 255.0

        output = model.predict(face_roi)[0]
        expression_index = np.argmax(output)
        return expression_labels[expression_index]

    return 'No Face'

# 메인 루프에 이 함수를 적용하는 부분은 이전 예제와 같습니다.

while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # 여기서 감정을 감지하는 로직이 필요합니다. (예시로 임의의 감정을 설정)
    expression_label = detect_emotion(frame)  # detect_emotion 함수를 이용하여 감정을 실시간으로 감지

    # 표정에 맞는 필터 적용
    frame = emotion_filters.apply_filter(frame, expression_label)

    cv2.imshow('Expression Recognition with Filters', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()


NameError: name 'cv2' is not defined